In [60]:
# Header from RIT notebook
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
##Setting up Google sdk environment
import os 
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/jhlu/.config/gcloud/application_default_credentials.json' 
os.environ['GCLOUD_PROJECT'] = 'som-nero-phi-nigam-starr' #som-nero-phi-rit
%load_ext google.cloud.bigquery
##Setting up BQ API
from google.cloud import bigquery
client=bigquery.Client()
import numpy as np
import pandas as pd
import datetime

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


/home/jhlu/.local/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [61]:
query = """
WITH
-- Collate patients receiving any recent acute respiratory illness (ARI) diagnosis
recentARI AS
( -- Look for any patient with an ARI, based on diagnosis code list suggested by CDC,
  --  that was recorded recently since testing was available (first date was 03/02/2020)
  -- Since Starr-OMOP jitters by +/- 31 days and we wish to cover all COVID tests, we use
  -- a start date of 01/31/2020
  SELECT
    condOccur.person_id, visit_occurrence_id, condition_start_DATE, condition_start_DATETIME, 
    condition_source_value, condition_source_concept_id,
    visit_concept_id, care_site_name
  FROM
    `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.condition_occurrence` AS condOccur 
  LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.visit_occurrence` USING (visit_occurrence_id)
  LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.care_site` USING (care_site_id)
  WHERE
    condition_start_DATE >= '2020-01-31' AND
    ( condition_source_value LIKE 'J00.%' OR
      condition_source_value LIKE 'J01.%' OR
      condition_source_value LIKE 'J02.%' OR
      condition_source_value LIKE 'J03.%' OR
      condition_source_value LIKE 'J04.%' OR
      condition_source_value LIKE 'J05.%' OR
      condition_source_value LIKE 'J06.%' OR
      condition_source_value LIKE 'J09.%' OR
      condition_source_value LIKE 'J10.%' OR
      condition_source_value LIKE 'J11.%' OR
      condition_source_value LIKE 'J12.%' OR
      condition_source_value LIKE 'J13.%' OR
      condition_source_value LIKE 'J14.%' OR
      condition_source_value LIKE 'J15.%' OR
      condition_source_value LIKE 'J16.%' OR
      condition_source_value LIKE 'J17.%' OR
      condition_source_value LIKE 'J18.%' OR
      condition_source_value LIKE 'J20.%' OR
      condition_source_value LIKE 'J21.%' OR
      condition_source_value LIKE 'J22' OR
      condition_source_value LIKE 'J80' OR
      condition_source_value LIKE 'A37.91' OR
      condition_source_value LIKE 'A37.01' OR
      condition_source_value LIKE 'A37.11' OR
      condition_source_value LIKE 'A37.81' OR
      condition_source_value LIKE 'A48.1' OR
      condition_source_value LIKE 'B25.0' OR
      condition_source_value LIKE 'B44.0' OR
      condition_source_value LIKE 'B97.4' OR
      condition_source_value LIKE 'U07.1'
    )
),

firstARIInfo AS
(
SELECT person_id, firstARIDateTime, firstARI_visit_concept_id, firstARI_care_site_name
FROM (
  SELECT 
    person_id, 
    condition_start_DATETIME AS firstARIDateTime,
    visit_concept_id AS firstARI_visit_concept_id,
    care_site_name AS firstARI_care_site_name,
    row_number() OVER (partition by person_id ORDER BY condition_start_DATETIME ASC) AS row_num
   FROM recentARI
) AS r
WHERE r.row_num=1
),

lastARIInfo AS
(
SELECT person_id, lastARIDateTime, lastARI_visit_concept_id, lastARI_care_site_name
FROM (
  SELECT 
    person_id, 
    condition_start_DATETIME AS lastARIDateTime,
    visit_concept_id AS lastARI_visit_concept_id,
    care_site_name AS lastARI_care_site_name,
    row_number() OVER (partition by person_id ORDER BY condition_start_DATETIME DESC) AS row_num
   FROM recentARI
) AS r
WHERE r.row_num=1
),

recentARIDateTimeRange AS
(
SELECT person_id, lastARIDateTime, lastARI_visit_concept_id, lastARI_care_site_name,
firstARIDateTime, firstARI_visit_concept_id, firstARI_care_site_name
FROM firstARIInfo
INNER JOIN lastARIInfo USING (person_id)
)
,

-- recentARIDateTimeRange AS
-- ( -- Look for only the last / most recent ARI occurrence date for each patient, in case one has multiple
--   SELECT
--     recentARI.person_id,
--     MIN(condition_start_DATETIME) AS firstARIDateTime, MAX(condition_start_DATETIME) AS lastARIDateTime
--   FROM recentARI
--   GROUP BY recentARI.person_id 
-- ),


-- Collate results on anyone receiving the SARS-CoV2 NAA Test Results
resultsSARSCoV2Tests AS
(
  SELECT meas.person_id, visit_occurrence_id, measurement_DATE, measurement_DATETIME, value_source_value, value_source_value IN ('Detected','Pos','Positive') AS detectedSARSCoV2, visit_concept_id, care_site_name
  FROM `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.measurement` as meas
  LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.visit_occurrence` USING (visit_occurrence_id)
  LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.care_site` USING (care_site_id)
  WHERE measurement_concept_id = 706170 -- SARS-CoV2 NAA Test Result
  AND meas.value_source_value is NOT NULL -- Only capture usable results and not extra descriptors
) ,
/* Note how often get null values
select count(*) nRecords, count(distinct person_id) as nPerson, count(distinct visit_occurrence_id) as nVisits, countif(value_source_value is not null) as nonNullValues
from resultsSARSCoV2Tests 
nRecords  42268
nPerson 14934
nVisits 21043
nonNullValues 21197
*/  

-- time, concept_ids, care_site_name from first test
firstTestInfo AS
(
SELECT person_id, firstSARSCoV2TestedDateTime, firstTest_visit_concept_id, firstTest_care_site_name
FROM (
  SELECT 
    person_id, 
    measurement_DATETIME AS firstSARSCoV2TestedDateTime,
    visit_concept_id AS firstTest_visit_concept_id,
    care_site_name AS firstTest_care_site_name,
    row_number() OVER (partition by person_id ORDER BY measurement_DATETIME ASC) AS row_num
   FROM resultsSARSCoV2Tests
) AS r
WHERE r.row_num=1
),

-- time, concept_ids, care_site_name from last test
lastTestInfo AS
(
SELECT person_id, lastSARSCoV2TestedDateTime, lastTest_visit_concept_id, lastTest_care_site_name
FROM (
  SELECT 
    person_id, 
    measurement_DATETIME AS lastSARSCoV2TestedDateTime,
    visit_concept_id AS lastTest_visit_concept_id,
    care_site_name AS lastTest_care_site_name,
    row_number() OVER (partition by person_id ORDER BY measurement_DATETIME DESC) AS row_num
   FROM resultsSARSCoV2Tests
) AS r
WHERE r.row_num=1
),

resultsSARSCoV2DateTimeRange AS( -- Accounting for multiple possible tests per patient, group by patient ID and collate date time ranges
  SELECT 
    person_id, 
    firstSARSCoV2TestedDateTime, firstTest_visit_concept_id, firstTest_care_site_name,
    lastSARSCoV2TestedDateTime, lastTest_visit_concept_id, lastTest_care_site_name,
    firstSARSCoV2DetectedDateTime, lastSARSCoV2DetectedDateTime
  FROM firstTestInfo
  INNER JOIN lastTestInfo USING (person_id)
  LEFT JOIN -- Left outer join, because not everyone who had a test will have a positive test result to join to
  (
    SELECT person_id, MIN(measurement_DATETIME) AS firstSARSCoV2DetectedDateTime, MAX(measurement_DATETIME) AS lastSARSCoV2DetectedDateTime
    FROM resultsSARSCoV2Tests
    WHERE resultsSARSCoV2Tests.detectedSARSCoV2
    GROUP BY person_id 
  ) AS firstLastDetectedSARSCoV2Tests
  USING (person_id)
  ),

-- At this point:
--  recentARIDateTimeRange has one row per patient (41,341 rows as of 2020-04-20 cdm_release_DATE)
--  resultsSARSCoV2DateTimeRange has one row per patient (14,838)
-- Each with first and last dates of occurrences of ARI (after initial "recent" cutoff date) orSARS-CoV2 test results (including timing of positive/detected result, null if no positive/detected results)
recentARIandSARSCoV2DateTimeRange AS
( -- Full outer join in both directions because some people with ARI diagnosis don't get SARS-CoV2 testing and vice versa
  SELECT *,
      DATETIME_DIFF( firstSARSCoV2TestedDateTime, firstARIDateTime, DAY ) daysFirstARItoFirstSARSCoV2Tested,
      DATETIME_DIFF( firstSARSCoV2TestedDateTime, lastARIDateTime, DAY ) daysLastARItoFirstSARSCoV2Tested,
      DATETIME_DIFF( lastSARSCoV2TestedDateTime, firstARIDateTime, DAY ) daysFirstARItoLastSARSCoV2Tested,
      DATETIME_DIFF( lastSARSCoV2TestedDateTime, lastARIDateTime, DAY ) daysLastARItoLastSARSCoV2Tested,

      DATETIME_DIFF( firstSARSCoV2DetectedDateTime, firstARIDateTime, DAY ) daysFirstARItoFirstSARSCoV2Detected,
      DATETIME_DIFF( firstSARSCoV2DetectedDateTime, lastARIDateTime, DAY ) daysLastARItoFirstSARSCoV2Detected,
      DATETIME_DIFF( lastSARSCoV2DetectedDateTime, firstARIDateTime, DAY ) daysFirstARItoLastSARSCoV2Detected,
      DATETIME_DIFF( lastSARSCoV2DetectedDateTime, lastARIDateTime, DAY ) daysLastARItoLastSARSCoV2Detected,
  FROM 
    recentARIDateTimeRange FULL JOIN 
    resultsSARSCoV2DateTimeRange USING (person_id)
  -- 4,091 inner join results (recent ARI diagnosis code AND SARS-CoV2 test results exist, but not necessarily in the correct datetime order)
  -- 52,088 full outer join results (recent ARI diagnosi code OR SARS-CoV2 test results)
)

SELECT 
    recentARIandSARSCoV2DateTimeRange.*,
    year_of_birth, DATE_DIFF('2020-02-01', DATE(birth_DATETIME), YEAR) ageAsOfFeb2020,
    person.gender_concept_id , genderConc.concept_name AS genderConcept,
    person.race_source_value, person.race_concept_id, raceConc.concept_name as raceConcept,
    person.ethnicity_source_value, person.ethnicity_concept_id, ethnicityConc.concept_name as ethnicityConcept
  FROM 
    recentARIandSARSCoV2DateTimeRange 
    JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.person` AS person USING (person_id) 
    LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.concept` AS genderConc ON (person.gender_concept_id = genderConc.concept_id)
    LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.concept` AS raceConc ON (person.race_concept_id = raceConc.concept_id)
    LEFT JOIN `som-rit-phi-starr-prod.starr_omop_cdm5_deid_latest.concept` AS ethnicityConc ON (person.ethnicity_concept_id = ethnicityConc.concept_id)
"""
query_job = client.query(query)
df = query_job.to_dataframe(progress_bar_type='tqdm')

Downloading: 100%|██████████| 52987/52987 [00:15<00:00, 3492.89rows/s]


In [62]:
df.columns

Index(['person_id', 'lastARIDateTime', 'lastARI_visit_concept_id',
       'lastARI_care_site_name', 'firstARIDateTime',
       'firstARI_visit_concept_id', 'firstARI_care_site_name',
       'firstSARSCoV2TestedDateTime', 'firstTest_visit_concept_id',
       'firstTest_care_site_name', 'lastSARSCoV2TestedDateTime',
       'lastTest_visit_concept_id', 'lastTest_care_site_name',
       'firstSARSCoV2DetectedDateTime', 'lastSARSCoV2DetectedDateTime',
       'daysFirstARItoFirstSARSCoV2Tested', 'daysLastARItoFirstSARSCoV2Tested',
       'daysFirstARItoLastSARSCoV2Tested', 'daysLastARItoLastSARSCoV2Tested',
       'daysFirstARItoFirstSARSCoV2Detected',
       'daysLastARItoFirstSARSCoV2Detected',
       'daysFirstARItoLastSARSCoV2Detected',
       'daysLastARItoLastSARSCoV2Detected', 'year_of_birth', 'ageAsOfFeb2020',
       'gender_concept_id', 'genderConcept', 'race_source_value',
       'race_concept_id', 'raceConcept', 'ethnicity_source_value',
       'ethnicity_concept_id', 'ethnicityCo

In [63]:
ARI = df['lastARIDateTime'].notnull().rename('ARI')
tested = df['lastSARSCoV2TestedDateTime'].notnull().rename('tested')
detected = df['lastSARSCoV2DetectedDateTime'].notnull().rename('detected')

in_time_range = lambda col : df[col].between(0, 60)
in_time_ranges = lambda col1, col2 : in_time_range(col1) | in_time_range(col2)

ARI_tested_60 = in_time_ranges('daysFirstARItoFirstSARSCoV2Tested', 'daysLastARItoFirstSARSCoV2Tested').rename('ARI_tested_60')
ARI_detected_60 = in_time_ranges('daysFirstARItoFirstSARSCoV2Detected', 'daysLastARItoFirstSARSCoV2Detected').rename('ARI_detected_60')

# age categorical
bins = [0, 5, 18, 50, 65, 75, 85, 100]
labels = ["[{:02d}, {:02d})".format(i, j) for i, j in zip(bins[:-1], bins[1:])]
age_cat = pd.cut(df['ageAsOfFeb2020'], bins, labels=labels).rename('age')

df['care_site'] = df['firstTest_care_site_name'].fillna(df['lastARI_care_site_name'])
df['visit_concept'] = df['firstTest_visit_concept_id'].fillna(df['lastARI_visit_concept_id'])

In [64]:
df_all = pd.concat([df, ARI, tested, detected, ARI_tested_60, ARI_detected_60, age_cat], axis=1)
df_all.head()

,person_id,lastARIDateTime,lastARI_visit_concept_id,lastARI_care_site_name,firstARIDateTime,firstARI_visit_concept_id,firstARI_care_site_name,firstSARSCoV2TestedDateTime,firstTest_visit_concept_id,firstTest_care_site_name,...,ethnicity_concept_id,ethnicityConcept,care_site,visit_concept,ARI,tested,detected,ARI_tested_60,ARI_detected_60,age
0,31343658,2020-05-09,9202.0,FAMILY MEDICINE OAKLAND,2020-05-09,9202.0,FAMILY MEDICINE OAKLAND,2020-05-10 10:03:00,9202.0,FAMILY MEDICINE OAKLAND,...,0,No matching concept,FAMILY MEDICINE OAKLAND,9202.0,True,True,False,True,False,"[50, 65)"
1,32118872,2020-03-07,0.0,INTERNAL MEDICINE BVMG HAYWARD,2020-03-02,9202.0,INTERNAL MEDICINE BVMG HAYWARD,NaT,NaN,None,...,0,No matching concept,INTERNAL MEDICINE BVMG HAYWARD,0.0,True,False,False,False,False,"[18, 50)"
2,31166808,2020-03-01,9202.0,FAMILY MEDICINE OAKLAND,2020-03-01,9202.0,FAMILY MEDICINE OAKLAND,NaT,NaN,None,...,0,No matching concept,FAMILY MEDICINE OAKLAND,9202.0,True,False,False,False,False,"[50, 65)"
3,43995102,2020-03-22,0.0,FAMILY MEDICINE ALMADEN SAN JOSE,2020-02-29,9202.0,FAMILY MEDICINE ALMADEN SAN JOSE,NaT,NaN,None,...,0,No matching concept,FAMILY MEDICINE ALMADEN SAN JOSE,0.0,True,False,False,False,False,"[65, 75)"
4,31406198,2020-04-07,9202.0,PEDS TRACY 4598 S TRACY,2020-04-07,9202.0,PEDS TRACY 4598 S TRACY,NaT,NaN,None,...,0,No matching concept,PEDS TRACY 4598 S TRACY,9202.0,True,False,False,False,False,"[05, 18)"


In [65]:
print('sums')
for elt in ['ARI', 'tested', 'detected', 'ARI_tested_60', 'ARI_detected_60']:
    df_all[elt] = df_all[elt].astype('int32')
    print(elt, np.sum(df_all[elt]))

sums
ARI 27043
tested 32532
detected 1502
ARI_tested_60 5356
ARI_detected_60 408


In [66]:
print('person_id is unique')
df_all['person_id'].is_unique
df_all.isnull().sum()

person_id is unique


person_id                                  0
lastARIDateTime                        25944
lastARI_visit_concept_id               26688
lastARI_care_site_name                 31259
firstARIDateTime                       25944
firstARI_visit_concept_id              26772
firstARI_care_site_name                31489
firstSARSCoV2TestedDateTime            20455
firstTest_visit_concept_id             20491
firstTest_care_site_name               24453
lastSARSCoV2TestedDateTime             20455
lastTest_visit_concept_id              20485
lastTest_care_site_name                24638
firstSARSCoV2DetectedDateTime          51485
lastSARSCoV2DetectedDateTime           51485
daysFirstARItoFirstSARSCoV2Tested      46399
daysLastARItoFirstSARSCoV2Tested       46399
daysFirstARItoLastSARSCoV2Tested       46399
daysLastARItoLastSARSCoV2Tested        46399
daysFirstARItoFirstSARSCoV2Detected    52393
daysLastARItoFirstSARSCoV2Detected     52393
daysFirstARItoLastSARSCoV2Detected     52393
daysLastAR

In [67]:
df_all['ethnicity_source_value'].value_counts().to_csv('ethnicity_source')
df_all['race_source_value'].value_counts().to_csv('race_source')
# for ind, grp in df_all.groupby('raceConcept'):
#     print(ind)
#     print(grp['race_source_value'].unique())
for ind, grp in df_all.groupby('ethnicityConcept'):
    print(ind)
    print(grp['ethnicity_source_value'].unique())

Hispanic or Latino
['Hispanic/Latino |' 'Hispanic/Latino | Not Hispanic'
 'Hispanic/Latino | Hispanic' 'Hispanic/Latino | Unknown'
 'Unknown | Hispanic' '| Hispanic' 'Hispanic/Latino | Declines to State'
 'Patient Refused | Hispanic']
No matching concept
['Unknown |' 'Patient Refused |' '| Declines to State' '|' '| Unknown'
 'Unknown | Declines to State' 'Patient Refused | Declines to State'
 'Unknown | Unknown' 'Patient Refused | Unknown']
Not Hispanic or Latino
['Non-Hispanic/Non-Latino |' 'Non-Hispanic/Non-Latino | Not Hispanic'
 'Non-Hispanic/Non-Latino | Hispanic' 'Non-Hispanic/Non-Latino | Unknown'
 '| Not Hispanic' 'Unknown | Not Hispanic'
 'Patient Refused | Not Hispanic'
 'Non-Hispanic/Non-Latino | Declines to State']


In [68]:
visit_token = 'visit_concept'
from IPython.display import display, HTML
def df_disp(df):
    display(HTML(df.to_html()))

def format_by_visit(ind_nm):
    tbl = pd.pivot_table(df_all, values=['ARI', 'tested', 'detected', 'ARI_tested_60', 'ARI_detected_60'], index=[ind_nm],
                            columns=[visit_token], aggfunc=np.sum)
    p_tbl = tbl.T.rename(columns=str).reset_index()
    p_tbl = p_tbl.rename({'level_0' : 'type'}, axis=1).sort_values([visit_token, 'type'])

    cols = p_tbl.columns.tolist()
    cols = [cols[1], cols[0]] + cols[2:]
    p_tbl = p_tbl[cols]
    df_disp(p_tbl)
    return p_tbl

def format_by_ind(ind_nm, write_folder="run_dumps"):
    tbl = pd.pivot_table(df_all, values=['ARI', 'tested', 'detected', 'ARI_tested_60', 'ARI_detected_60'], index=[ind_nm],
                            aggfunc=np.sum)
    p_tbl = tbl.T
    df_disp(p_tbl)
    return p_tbl

In [69]:
care_site_tok = 'care_site'
save_care_site_csv = lambda s, tok : s.value_counts().to_csv('care_site_counts_' + tok + '_visit.csv')
save_care_site_csv(df_all[care_site_tok], 'all')
save_care_site_csv(df_all[care_site_tok][df_all['ARI'] == 1], 'ARI')
save_care_site_csv(df_all[care_site_tok][df_all['tested'] == 1], 'tested')
save_care_site_csv(df_all[care_site_tok][df_all['detected'] == 1], 'detected')

In [70]:
for elt in ['age', 'ethnicityConcept', 'raceConcept', 'genderConcept']:
    format_by_visit(elt)
    format_by_ind(elt)

age,visit_concept,type,"[00, 05)","[05, 18)","[18, 50)","[50, 65)","[65, 75)","[75, 85)","[85, 100)"
0,0.0,ARI,965.0,1466.0,3141.0,1393.0,717.0,319.0,66.0
4,0.0,ARI_detected_60,4.0,6.0,155.0,50.0,19.0,5.0,1.0
8,0.0,ARI_tested_60,113.0,182.0,1975.0,711.0,340.0,127.0,20.0
12,0.0,detected,10.0,28.0,614.0,296.0,145.0,92.0,37.0
16,0.0,tested,585.0,867.0,15890.0,4785.0,2868.0,1429.0,354.0
1,9201.0,ARI,661.0,657.0,1703.0,780.0,399.0,275.0,83.0
5,9201.0,ARI_detected_60,1.0,2.0,65.0,32.0,13.0,13.0,5.0
9,9201.0,ARI_tested_60,60.0,45.0,495.0,293.0,151.0,107.0,33.0
13,9201.0,detected,1.0,3.0,94.0,44.0,20.0,24.0,5.0
17,9201.0,tested,156.0,174.0,1492.0,1074.0,606.0,417.0,123.0


age,"[00, 05)","[05, 18)","[18, 50)","[50, 65)","[65, 75)","[75, 85)","[85, 100)"
ARI,4159,5522,9468,4289,2129,1106,283
ARI_detected_60,6,9,233,100,35,18,6
ARI_tested_60,182,242,2798,1239,563,262,58
detected,12,35,737,377,175,121,44
tested,756,1097,18123,6387,3694,1936,493


ethnicityConcept,visit_concept,type,Hispanic or Latino,No matching concept,Not Hispanic or Latino
0,0.0,ARI,1027.0,2138.0,4927.0
4,0.0,ARI_detected_60,60.0,76.0,105.0
8,0.0,ARI_tested_60,500.0,679.0,2298.0
12,0.0,detected,175.0,762.0,286.0
16,0.0,tested,2728.0,10392.0,13694.0
1,9201.0,ARI,1184.0,55.0,3344.0
5,9201.0,ARI_detected_60,60.0,1.0,70.0
9,9201.0,ARI_tested_60,258.0,9.0,920.0
13,9201.0,detected,75.0,1.0,115.0
17,9201.0,tested,861.0,49.0,3142.0


ethnicityConcept,Hispanic or Latino,No matching concept,Not Hispanic or Latino
ARI,3991,5057,17995
ARI_detected_60,125,80,203
ARI_tested_60,842,790,3724
detected,264,771,467
tested,3799,10671,18062


raceConcept,visit_concept,type,American Indian or Alaska Native,Asian,Black or African American,Native Hawaiian or Other Pacific Islander,No matching concept,White
0,0.0,ARI,23.0,1195.0,243.0,73.0,3269.0,3289.0
4,0.0,ARI_detected_60,0.0,31.0,6.0,3.0,126.0,75.0
8,0.0,ARI_tested_60,6.0,540.0,114.0,33.0,1216.0,1568.0
12,0.0,detected,2.0,69.0,13.0,4.0,926.0,209.0
16,0.0,tested,59.0,3581.0,677.0,255.0,13138.0,9104.0
1,9201.0,ARI,10.0,822.0,241.0,103.0,1320.0,2087.0
5,9201.0,ARI_detected_60,0.0,20.0,4.0,2.0,64.0,41.0
9,9201.0,ARI_tested_60,2.0,199.0,53.0,27.0,341.0,565.0
13,9201.0,detected,0.0,29.0,6.0,6.0,81.0,69.0
17,9201.0,tested,12.0,597.0,229.0,95.0,1084.0,2035.0


raceConcept,American Indian or Alaska Native,Asian,Black or African American,Native Hawaiian or Other Pacific Islander,No matching concept,White
ARI,66,4791,1005,301,9149,11731
ARI_detected_60,0,55,12,6,197,138
ARI_tested_60,12,866,189,67,1732,2490
detected,2,110,23,12,1029,326
tested,82,4458,974,366,14656,11996


genderConcept,visit_concept,type,FEMALE,MALE,No matching concept
0,0.0,ARI,4667.0,3422.0,3.0
4,0.0,ARI_detected_60,123.0,118.0,0.0
8,0.0,ARI_tested_60,2195.0,1280.0,2.0
12,0.0,detected,582.0,641.0,0.0
16,0.0,tested,17997.0,8800.0,17.0
1,9201.0,ARI,2273.0,2309.0,1.0
5,9201.0,ARI_detected_60,51.0,80.0,0.0
9,9201.0,ARI_tested_60,541.0,645.0,1.0
13,9201.0,detected,74.0,117.0,0.0
17,9201.0,tested,1858.0,2193.0,1.0


genderConcept,FEMALE,MALE,No matching concept
ARI,14768,12267,8
ARI_detected_60,196,212,0
ARI_tested_60,3153,2199,4
detected,699,803,0
tested,20798,11715,19


In [71]:
for elt in ['age', 'ethnicity_source_value', 'race_source_value', 'genderConcept']:
    format_by_visit(elt)
    format_by_ind(elt)
    

age,visit_concept,type,"[00, 05)","[05, 18)","[18, 50)","[50, 65)","[65, 75)","[75, 85)","[85, 100)"
0,0.0,ARI,965.0,1466.0,3141.0,1393.0,717.0,319.0,66.0
4,0.0,ARI_detected_60,4.0,6.0,155.0,50.0,19.0,5.0,1.0
8,0.0,ARI_tested_60,113.0,182.0,1975.0,711.0,340.0,127.0,20.0
12,0.0,detected,10.0,28.0,614.0,296.0,145.0,92.0,37.0
16,0.0,tested,585.0,867.0,15890.0,4785.0,2868.0,1429.0,354.0
1,9201.0,ARI,661.0,657.0,1703.0,780.0,399.0,275.0,83.0
5,9201.0,ARI_detected_60,1.0,2.0,65.0,32.0,13.0,13.0,5.0
9,9201.0,ARI_tested_60,60.0,45.0,495.0,293.0,151.0,107.0,33.0
13,9201.0,detected,1.0,3.0,94.0,44.0,20.0,24.0,5.0
17,9201.0,tested,156.0,174.0,1492.0,1074.0,606.0,417.0,123.0


age,"[00, 05)","[05, 18)","[18, 50)","[50, 65)","[65, 75)","[75, 85)","[85, 100)"
ARI,4159,5522,9468,4289,2129,1106,283
ARI_detected_60,6,9,233,100,35,18,6
ARI_tested_60,182,242,2798,1239,563,262,58
detected,12,35,737,377,175,121,44
tested,756,1097,18123,6387,3694,1936,493


ethnicity_source_value,visit_concept,type,Hispanic/Latino |,Hispanic/Latino | Declines to State,Hispanic/Latino | Hispanic,Hispanic/Latino | Not Hispanic,Hispanic/Latino | Unknown,Non-Hispanic/Non-Latino |,Non-Hispanic/Non-Latino | Declines to State,Non-Hispanic/Non-Latino | Hispanic,Non-Hispanic/Non-Latino | Not Hispanic,Non-Hispanic/Non-Latino | Unknown,Patient Refused |,Patient Refused | Declines to State,Patient Refused | Hispanic,Patient Refused | Not Hispanic,Patient Refused | Unknown,Unknown |,Unknown | Declines to State,Unknown | Hispanic,Unknown | Not Hispanic,Unknown | Unknown,|,| Declines to State,| Hispanic,| Not Hispanic,| Unknown
0,0.0,ARI,506.0,3.0,239.0,67.0,29.0,2837.0,26.0,54.0,934.0,240.0,137.0,5.0,1.0,12.0,5.0,334.0,11.0,6.0,76.0,47.0,943.0,126.0,176.0,748.0,530.0
4,0.0,ARI_detected_60,34.0,0.0,21.0,2.0,0.0,81.0,1.0,1.0,14.0,2.0,1.0,0.0,0.0,0.0,1.0,12.0,0.0,0.0,3.0,1.0,60.0,0.0,3.0,3.0,1.0
8,0.0,ARI_tested_60,311.0,0.0,129.0,33.0,7.0,1668.0,9.0,26.0,444.0,81.0,83.0,1.0,0.0,6.0,2.0,226.0,0.0,0.0,22.0,15.0,328.0,4.0,20.0,42.0,20.0
12,0.0,detected,122.0,0.0,44.0,2.0,2.0,229.0,2.0,1.0,37.0,6.0,6.0,0.0,0.0,0.0,1.0,69.0,0.0,0.0,5.0,2.0,683.0,0.0,5.0,6.0,1.0
16,0.0,tested,1680.0,6.0,658.0,188.0,42.0,9948.0,46.0,133.0,2567.0,414.0,345.0,3.0,9.0,58.0,8.0,1408.0,8.0,32.0,190.0,65.0,8423.0,21.0,113.0,338.0,111.0
1,9201.0,ARI,556.0,3.0,482.0,59.0,61.0,2513.0,17.0,58.0,558.0,168.0,28.0,NaN,1.0,2.0,2.0,20.0,1.0,0.0,3.0,1.0,1.0,NaN,22.0,25.0,2.0
5,9201.0,ARI_detected_60,45.0,0.0,14.0,0.0,1.0,65.0,0.0,1.0,4.0,0.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
9,9201.0,ARI_tested_60,171.0,0.0,65.0,10.0,7.0,764.0,2.0,7.0,117.0,24.0,7.0,NaN,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,4.0,5.0,1.0
13,9201.0,detected,56.0,0.0,18.0,0.0,1.0,104.0,0.0,2.0,8.0,1.0,1.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0
17,9201.0,tested,598.0,1.0,194.0,24.0,15.0,2626.0,5.0,23.0,382.0,70.0,29.0,NaN,1.0,2.0,1.0,13.0,1.0,3.0,9.0,0.0,1.0,NaN,25.0,25.0,4.0


ethnicity_source_value,Hispanic/Latino |,Hispanic/Latino | Declines to State,Hispanic/Latino | Hispanic,Hispanic/Latino | Not Hispanic,Hispanic/Latino | Unknown,Non-Hispanic/Non-Latino |,Non-Hispanic/Non-Latino | Declines to State,Non-Hispanic/Non-Latino | Hispanic,Non-Hispanic/Non-Latino | Not Hispanic,Non-Hispanic/Non-Latino | Unknown,Patient Refused |,Patient Refused | Declines to State,Patient Refused | Hispanic,Patient Refused | Not Hispanic,Patient Refused | Unknown,Unknown |,Unknown | Declines to State,Unknown | Hispanic,Unknown | Not Hispanic,Unknown | Unknown,|,| Declines to State,| Hispanic,| Not Hispanic,| Unknown
ARI,1983,12,1019,236,170,11069,122,169,3091,947,631,12,14,77,29,908,25,27,224,131,983,395,530,2296,1943
ARI_detected_60,83,0,36,2,1,167,1,2,22,4,2,0,0,0,1,14,0,0,4,1,61,0,3,3,1
ARI_tested_60,543,0,209,48,16,2840,12,34,636,119,123,1,1,9,4,289,0,1,27,19,329,4,24,47,21
detected,190,0,63,2,4,386,2,3,53,11,8,0,0,0,1,75,0,0,6,2,684,0,5,6,1
tested,2447,7,875,222,62,13578,52,162,3119,515,441,3,10,64,11,1568,10,37,207,70,8432,21,139,365,115


race_source_value,visit_concept,type,American Indian or Alaska Native |,American Indian or Alaska Native | American Indian or Alaska Native,American Indian or Alaska Native | Asian,American Indian or Alaska Native | Black or African American,American Indian or Alaska Native | Other,American Indian or Alaska Native | Unknown,American Indian or Alaska Native | White or Caucasian,Asian |,Asian | American Indian or Alaska Native,Asian | Asian,Asian | Black or African American,Asian | Declines to State,Asian | Native Hawaiian or Other Pacific Islander,Asian | Other,Asian | Unknown,Asian | White or Caucasian,"Asian, Hispanic |","Asian, non-Hispanic |","Asian, non-Hispanic | Asian","Asian, non-Hispanic | White or Caucasian",Black or African American |,Black or African American | American Indian or Alaska Native,Black or African American | Asian,Black or African American | Black or African American,Black or African American | Declines to State,Black or African American | Other,Black or African American | Unknown,Black or African American | White or Caucasian,"Black, Hispanic |","Black, non-Hispanic |","Black, non-Hispanic | Black or African American","Native American, Hispanic |",Native Hawaiian or Other Pacific Islander |,Native Hawaiian or Other Pacific Islander | Asian,Native Hawaiian or Other Pacific Islander | Native Hawaiian or Other Pacific Islander,Native Hawaiian or Other Pacific Islander | Other,Native Hawaiian or Other Pacific Islander | Unknown,Native Hawaiian or Other Pacific Islander | White or Caucasian,Other |,Other | American Indian or Alaska Native,Other | Asian,Other | Black or African American,Other | Declines to State,Other | Native Hawaiian or Other Pacific Islander,Other | Other,Other | Unknown,Other | White or Caucasian,"Other, Hispanic |","Other, Hispanic | Asian","Other, Hispanic | Black or African American","Other, Hispanic | Declines to State","Other, Hispanic | Native Hawaiian or Other Pacific Islander","Other, Hispanic | Other","Other, Hispanic | Unknown","Other, Hispanic | White or Caucasian","Other, non-Hispanic |","Other, non-Hispanic | Asian","Other, non-Hispanic | Other","Other, non-Hispanic | White or Caucasian","Pacific Islander, non-Hispanic |","Pacific Islander, non-Hispanic | Native Hawaiian or Other Pacific Islander",Patient Refused |,Patient Refused | American Indian or Alaska Native,Patient Refused | Asian,Patient Refused | Black or African American,Patient Refused | Declines to State,Patient Refused | Native Hawaiian or Other Pacific Islander,Patient Refused | Other,Patient Refused | Unknown,Patient Refused | White or Caucasian,Race and Ethnicity Unknown |,Race and Ethnicity Unknown | American Indian or Alaska Native,Race and Ethnicity Unknown | Asian,Race and Ethnicity Unknown | Black or African American,Race and Ethnicity Unknown | Declines to State,Race and Ethnicity Unknown | Other,Race and Ethnicity Unknown | Unknown,Race and Ethnicity Unknown | White or Caucasian,Unknown |,Unknown | American Indian or Alaska Native,Unknown | Asian,Unknown | Black or African American,Unknown | Declines to State,Unknown | Native Hawaiian or Other Pacific Islander,Unknown | Other,Unknown | Unknown,Unknown | White or Caucasian,White |,White | American Indian or Alaska Native,White | Asian,White | Black or African American,White | Declines to State,White | Native Hawaiian or Other Pacific Islander,White | Other,White | Unknown,White | White or Caucasian,"White, Hispanic |","White, Hispanic | Other","White, Hispanic | White or Caucasian","White, non-Hispanic |","White, non-Hispanic | Declines to State","White, non-Hispanic | Other","White, non-Hispanic | Unknown","White, non-Hispanic | White or Caucasian",|,| American Indian or Alaska Native,| Asian,| Black or African American,| Declines to State,| Native Hawaiian or Other Pacific Islander,| Other,| Unknown,| White or Caucasian
0,0.0,ARI,17.0,1.0,2.0,0.0,1.0,NaN,1.0,583.0,NaN,180.0,0.0,7.0,1.0,23.0,60.0,17.0,1.0,2.0,6.0,1.0,151.0,1.0,0.0,30.0,1.0,2.0,5.0,4.0

race_source_value,American Indian or Alaska Native |,American Indian or Alaska Native | American Indian or Alaska Native,American Indian or Alaska Native | Asian,American Indian or Alaska Native | Black or African American,American Indian or Alaska Native | Other,American Indian or Alaska Native | Unknown,American Indian or Alaska Native | White or Caucasian,Asian |,Asian | American Indian or Alaska Native,Asian | Asian,Asian | Black or African American,Asian | Declines to State,Asian | Native Hawaiian or Other Pacific Islander,Asian | Other,Asian | Unknown,Asian | White or Caucasian,"Asian, Hispanic |","Asian, non-Hispanic |","Asian, non-Hispanic | Asian","Asian, non-Hispanic | White or Caucasian",Black or African American |,Black or African American | American Indian or Alaska Native,Black or African American | Asian,Black or African American | Black or African American,Black or African American | Declines to State,Black or African American | Other,Black or African American | Unknown,Black or African American | White or Caucasian,"Black, Hispanic |","Black, non-Hispanic |","Black, non-Hispanic | Black or African American","Native American, Hispanic |",Native Hawaiian or Other Pacific Islander |,Native Hawaiian or Other Pacific Islander | Asian,Native Hawaiian or Other Pacific Islander | Native Hawaiian or Other Pacific Islander,Native Hawaiian or Other Pacific Islander | Other,Native Hawaiian or Other Pacific Islander | Unknown,Native Hawaiian or Other Pacific Islander | White or Caucasian,Other |,Other | American Indian or Alaska Native,Other | Asian,Other | Black or African American,Other | Declines to State,Other | Native Hawaiian or Other Pacific Islander,Other | Other,Other | Unknown,Other | White or Caucasian,"Other, Hispanic |","Other, Hispanic | Asian","Other, Hispanic | Black or African American","Other, Hispanic | Declines to State","Other, Hispanic | Native Hawaiian or Other Pacific Islander","Other, Hispanic | Other","Other, Hispanic | Unknown","Other, Hispanic | White or Caucasian","Other, non-Hispanic |","Other, non-Hispanic | Asian","Other, non-Hispanic | Other","Other, non-Hispanic | White or Caucasian","Pacific Islander, non-Hispanic |","Pacific Islander, non-Hispanic | Native Hawaiian or Other Pacific Islander",Patient Refused |,Patient Refused | American Indian or Alaska Native,Patient Refused | Asian,Patient Refused | Black or African American,Patient Refused | Declines to State,Patient Refused | Native Hawaiian or Other Pacific Islander,Patient Refused | Other,Patient Refused | Unknown,Patient Refused | White or Caucasian,Race and Ethnicity Unknown |,Race and Ethnicity Unknown | American Indian or Alaska Native,Race and Ethnicity Unknown | Asian,Race and Ethnicity Unknown | Black or African American,Race and Ethnicity Unknown | Declines to State,Race and Ethnicity Unknown | Other,Race and Ethnicity Unknown | Unknown,Race and Ethnicity Unknown | White or Caucasian,Unknown |,Unknown | American Indian or Alaska Native,Unknown | Asian,Unknown | Black or African American,Unknown | Declines to State,Unknown | Native Hawaiian or Other Pacific Islander,Unknown | Other,Unknown | Unknown,Unknown | White or Caucasian,White |,White | American Indian or Alaska Native,White | Asian,White | Black or African American,White | Declines to State,White | Native Hawaiian or Other Pacific Islander,White | Other,White | Unknown,White | White or Caucasian,"White, Hispanic |","White, Hispanic | Other","White, Hispanic | White or Caucasian","White, non-Hispanic |","White, non-Hispanic | Declines to State","White, non-Hispanic | Other","White, non-Hispanic | Unknown","White, non-Hispanic | White or Caucasian",|,| American Indian or Alaska Native,| Asian,| Black or African American,| Declines to State,| Native Hawaiian or Other Pacific Islander,| Other,| Unknown,| White or Caucasian
ARI,40,1,2,0,2,1,4,2656,2,705,0,37,8,88,218,48,1,6,15,1,636,2,2,135,2,26,33,7,0,4,6,0,147,45,47,9,12,9,2428,5,161,12,27,10,771,313,487,8,2,1,3,2,2,5,6,

genderConcept,visit_concept,type,FEMALE,MALE,No matching concept
0,0.0,ARI,4667.0,3422.0,3.0
4,0.0,ARI_detected_60,123.0,118.0,0.0
8,0.0,ARI_tested_60,2195.0,1280.0,2.0
12,0.0,detected,582.0,641.0,0.0
16,0.0,tested,17997.0,8800.0,17.0
1,9201.0,ARI,2273.0,2309.0,1.0
5,9201.0,ARI_detected_60,51.0,80.0,0.0
9,9201.0,ARI_tested_60,541.0,645.0,1.0
13,9201.0,detected,74.0,117.0,0.0
17,9201.0,tested,1858.0,2193.0,1.0


genderConcept,FEMALE,MALE,No matching concept
ARI,14768,12267,8
ARI_detected_60,196,212,0
ARI_tested_60,3153,2199,4
detected,699,803,0
tested,20798,11715,19


In [77]:
# change this to true if you reall ywant
export_report = True
export_files = []

if export_report:
    run_date = datetime.datetime.today().strftime("%y-%m-%d")
    print("Run date:", run_date)
    
    out_folder = os.path.join("stanford_starr-omop_covid_mmwr", run_date)
    os.makedirs(out_folder, exist_ok=True)
    
    outfile = os.path.join(out_folder, "df_all_" + run_date + ".csv")
    df_all.to_csv(outfile, index=False)
    
    print("Writing to ", out_folder)
    print(outfile, "written")
    
    for elt in ['age', 'ethnicity_source_value', 'race_source_value', 'genderConcept']:
        
        outfile = os.path.join(out_folder, elt +"_by_visit_" + run_date + ".csv")
        elt_visit = format_by_visit(elt)
        
        elt_visit.to_csv(outfile, index=True)
        print(outfile, "Written")
        
        
        outfile = os.path.join(out_folder, elt + "_by_ind_" + run_date + ".csv")
        elt_ind = format_by_ind(elt)
        
        elt_ind.to_csv(outfile, index=True)
        print(outfile, "Written")
        
        export_files.append(outfile)
        
print("Files to export", export_files)
        
        
    
    

Run date: 20-05-20
Writing to  stanford_starr-omop_covid_mmwr/20-05-20
stanford_starr-omop_covid_mmwr/20-05-20/df_all_20-05-20.csv written


age,visit_concept,type,"[00, 05)","[05, 18)","[18, 50)","[50, 65)","[65, 75)","[75, 85)","[85, 100)"
0,0.0,ARI,965.0,1466.0,3141.0,1393.0,717.0,319.0,66.0
4,0.0,ARI_detected_60,4.0,6.0,155.0,50.0,19.0,5.0,1.0
8,0.0,ARI_tested_60,113.0,182.0,1975.0,711.0,340.0,127.0,20.0
12,0.0,detected,10.0,28.0,614.0,296.0,145.0,92.0,37.0
16,0.0,tested,585.0,867.0,15890.0,4785.0,2868.0,1429.0,354.0
1,9201.0,ARI,661.0,657.0,1703.0,780.0,399.0,275.0,83.0
5,9201.0,ARI_detected_60,1.0,2.0,65.0,32.0,13.0,13.0,5.0
9,9201.0,ARI_tested_60,60.0,45.0,495.0,293.0,151.0,107.0,33.0
13,9201.0,detected,1.0,3.0,94.0,44.0,20.0,24.0,5.0
17,9201.0,tested,156.0,174.0,1492.0,1074.0,606.0,417.0,123.0


stanford_starr-omop_covid_mmwr/20-05-20/age_by_visit_20-05-20.csv Written


age,"[00, 05)","[05, 18)","[18, 50)","[50, 65)","[65, 75)","[75, 85)","[85, 100)"
ARI,4159,5522,9468,4289,2129,1106,283
ARI_detected_60,6,9,233,100,35,18,6
ARI_tested_60,182,242,2798,1239,563,262,58
detected,12,35,737,377,175,121,44
tested,756,1097,18123,6387,3694,1936,493


stanford_starr-omop_covid_mmwr/20-05-20/age_by_ind_20-05-20.csv Written


...

stanford_starr-omop_covid_mmwr/20-05-20/race_source_value_by_ind_20-05-20.csv Written


genderConcept,visit_concept,type,FEMALE,MALE,No matching concept
0,0.0,ARI,4667.0,3422.0,3.0
4,0.0,ARI_detected_60,123.0,118.0,0.0
8,0.0,ARI_tested_60,2195.0,1280.0,2.0
12,0.0,detected,582.0,641.0,0.0
16,0.0,tested,17997.0,8800.0,17.0
1,9201.0,ARI,2273.0,2309.0,1.0
5,9201.0,ARI_detected_60,51.0,80.0,0.0
9,9201.0,ARI_tested_60,541.0,645.0,1.0
13,9201.0,detected,74.0,117.0,0.0
17,9201.0,tested,1858.0,2193.0,1.0


stanford_starr-omop_covid_mmwr/20-05-20/genderConcept_by_visit_20-05-20.csv Written


genderConcept,FEMALE,MALE,No matching concept
ARI,14768,12267,8
ARI_detected_60,196,212,0
ARI_tested_60,3153,2199,4
detected,699,803,0
tested,20798,11715,19


stanford_starr-omop_covid_mmwr/20-05-20/genderConcept_by_ind_20-05-20.csv Written
Files to export ['stanford_starr-omop_covid_mmwr/20-05-20/age_by_ind_20-05-20.csv', 'stanford_starr-omop_covid_mmwr/20-05-20/ethnicity_source_value_by_ind_20-05-20.csv', 'stanford_starr-omop_covid_mmwr/20-05-20/race_source_value_by_ind_20-05-20.csv', 'stanford_starr-omop_covid_mmwr/20-05-20/genderConcept_by_ind_20-05-20.csv']


In [78]:
print("Are you sure you want to export?")
for export_file in export_files:
    print(export_file)

Are you sure you want to export?
stanford_starr-omop_covid_mmwr/20-05-20/age_by_ind_20-05-20.csv
stanford_starr-omop_covid_mmwr/20-05-20/ethnicity_source_value_by_ind_20-05-20.csv
stanford_starr-omop_covid_mmwr/20-05-20/race_source_value_by_ind_20-05-20.csv
stanford_starr-omop_covid_mmwr/20-05-20/genderConcept_by_ind_20-05-20.csv


In [79]:
with open("export_list.txt", 'w') as f:
    for export_file in export_files:
        f.write(export_file + "\n")
        
        
print("Are you sure you want to export?")
!head export_list.txt

Are you sure you want to export?
stanford_starr-omop_covid_mmwr/20-05-20/age_by_ind_20-05-20.csv
stanford_starr-omop_covid_mmwr/20-05-20/ethnicity_source_value_by_ind_20-05-20.csv
stanford_starr-omop_covid_mmwr/20-05-20/race_source_value_by_ind_20-05-20.csv
stanford_starr-omop_covid_mmwr/20-05-20/genderConcept_by_ind_20-05-20.csv


In [81]:
print("Now that export_list is updated, open a terminal and do:")
print('cd "/home/jhlu/CDSS/scripts/Covid19 CDC/"')
print('./git_export.sh')

Now that export_list is updated, open a terminal and do:
cd "/home/jhlu/CDSS/scripts/Covid19 CDC/"
./git_export.sh
